<a href="https://colab.research.google.com/github/SherbyRobotics/pyro/blob/colab/examples/notebooks/5DoF_robot_arm_nullspace_kinematic_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kinematic control of redundant robotic arm



## Importing Librairies

In [ ]:
# Loading a robotic toolbox
!git clone https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')
import pyro

# Generic python tools
import numpy as np
from IPython import display

# Defining the robot arm

Here we load a already defined class from the library including the dynamic equations defining the robot arm behavior.



In [ ]:
from pyro.dynamic import manipulator

# Dynamic model (inputs are motor torques)
torque_controlled_robot    = manipulator.FiveLinkPlanarManipulator()

# Kinematic only model (inputs are motor velocities)
speed_controlled_robot  = manipulator.SpeedControlledManipulator.from_manipulator( torque_controlled_robot )

robot = speed_controlled_robot # For this exercise, we will only use the kinematic model

## Showing the robotbehavior with constant velocity inputs

Here we run a simulation of the robot arm with no controllers:

In [ ]:
# robot initial states [ joint 1 angle (rad),  joint 2 angle (rad), joint 1 velocity (rad/sec),  joint 1 velocity (rad/sec)]
robot.x0 = np.zeros(5)

# robot constant inputs
robot.ubar = np.array([ 0.5, 0.5, 0.5, 0.5, 0.5]) # Constant joint velocities

# run the simulation
robot.compute_trajectory( tf = 6 )

# Animate and display the simulation
ani  = robot.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

# Basic kinematic controller

In [ ]:
from pyro.control import robotcontrollers

# Controller
ctl       = robotcontrollers.EndEffectorKinematicController( robot )
ctl.rbar  = np.array([1.0,1.0]) # target effector position
ctl.gains = np.array([2.0,2.0]) # gains

In [ ]:
# Closed-loop dynamics
cl_sys  = ctl + robot

# Initial config
cl_sys.x0 = np.array([0.1,0.1,0.1,0.1,0.1])  

# Simulation
cl_sys.compute_trajectory( tf = 5 )

In [ ]:
# Animate the simulation
ani = cl_sys.generate_simulation_html_video()
html = display.HTML(ani)
display.display(html)

In [ ]:
#cl_sys.plot_trajectory('x')

In [ ]:
#cl_sys.plot_trajectory('u')

# Nullspace kinematic controller

In [ ]:
# Controller
ctl = robotcontrollers.EndEffectorKinematicControllerWithNullSpaceTask( robot )

# Main objective
ctl.rbar  = np.array([1.0,1.0])
ctl.gains = np.array([1.0,1.0])

# Secondary objective
ctl.q_d         = np.array([-1,-1,-1,-1,-1])
ctl.gains_null  = np.array([10,10,10,10,10])

In [ ]:
# Closed-loop dynamics
cl_sys  = ctl + robot

# Initial config
cl_sys.x0 = np.array([0.1,0.1,0.1,0.1,0.1])  

# Simulation
cl_sys.compute_trajectory( tf = 5 )

In [ ]:
# Animate the simulation
ani = cl_sys.generate_simulation_html_video()
html = display.HTML(ani)
display.display(html)

# Custom Kinematic controller

Here bellow you can write your own custom kinematic control law, i.e. computing the joint velocity based on actual joint position and time. Here the framework is already set in place for you to speed-up the implementation process. The following variable are pre-computed and available for you to use in the feedback law:

*   The joint angles $q$ 
*   The time $t$

Furthermore, based on a kinematic robot model the following properties are also pre-computed:
*   The actual end-effector position $r$
*   The Jacobian $J(q)$



In [ ]:
from pyro.control  import robotcontrollers

class CustomKinematicController( robotcontrollers.EndEffectorKinematicController ) :

    #############################
    def c( self , y , r , t ):
        """ 
        Feedback static computation u = c(y,r,t)
        
        INPUTS
        y  : sensor signal vector     p x 1
        r  : reference signal vector  k x 1
        t  : time                     1 x 1
        
        OUTPUTS
        dq  : joint velocity vector   m x 1
        
        """

        # Feedback from sensors
        q = y                  # Joint angles
        
        # Pre-computed values based on the robot kinematic model
        r      = self.fwd_kin( q )    # End-effector postion
        J      = self.J( q )          # Jacobian computation
        J_pinv = np.linalg.pinv( J )  # Pseudo Inverse
        Null_p = np.identity( self.dof ) - np.dot(J_pinv,J) # Nullspace projection Matrix

        ##############################
        # YOUR CODE BELLOW !!
        ##############################

        dq = Null_p @ np.array([1.0,0,0,0,0])
        
        return dq

In [ ]:
ctl = CustomKinematicController( robot )

## Simulating the robot in closed-loop

Here we run a simulation of the robot with the controller.

In [ ]:
# Create the closed-loop system
robot_with_controller = ctl + robot

# Run the simulation
robot_with_controller.x0 = np.array([0.1,0.2,0.2,0.2,0.2]) 
robot_with_controller.compute_trajectory( tf = 3 )

In [ ]:
# Animate the simulation
ani = robot_with_controller.generate_simulation_html_video()
html = display.HTML(ani)
display.display(html)

In [ ]:
# Plot systems states
robot_with_controller.plot_trajectory('x')

In [ ]:
# Plot control inputs
robot_with_controller.plot_trajectory('u')